Здесь идет добавление всех нужных библиотек и команд

In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import numpy as np
%matplotlib inline
print("Setup Complite")

Setup Complite


In [23]:
#Path of the file to read
filepath = "health_lifestyle_classification.csv"
#Read the file into a variable df
df = pd.read_csv(filepath)


В переменной filepath хранится путь до датасета. В переменной insurance_data хранится дата фрейм по нашему датасету

In [24]:
delete_columns = [
    'screen_time',
    'family_history',
    'mental_health_score',
    'occupation',
    'mental_health_support',
    'device_usage',
    'healthcare_access',
    'insurance',
    'pet_owner',
    'height',
    'weight',
    'waist_size',
    'bmi',
    'physical_activity',
    'education_level',
    'job_type',
    'income',
    'electrolyte_level',
    'gene_marker_flag',
    'environmental_risk_score',
    'daily_supplement_dosage'
]

for column in delete_columns:
    df = df.drop(column, axis=1)



В датасете более 100+ тысяч строк и 41 признак. Выберем необходимое и отсеем лишнее.

Целевым признаком для предсказания будет target (Бинарная классификация): healthy (здоров) / diseased (болен)

Из числовых признаков возьмём:
- age (Возраст)
- bmi (Индекс массы тела)
- blood pressure
- cholesterol
- glucose
- insulin
- calorie intake
- sugar intake
- water intake
- stress level
- heart rate
- sleep hours
- work hours
- daily steps

Из категориальных признаков возьмём:
- gender
- diet type
- sleep quality
- sunlight exposure
- caffeine intake
- exercise type
- meals per day
- alchohol consumption
- smoking level

Таким образом были убраны следующие признаки:
- screen time
- family history
- mental health score
- occupation
- mental health support
- device usage
- healthcare access
- insurance
- pet owner
- height
- weight
- waist size
- bmi ?*
- physical activity
- education level
- job type
- income
- electrolyte level
- gene marker flag
- enviromnent risk score
- daily supplement dosage


In [25]:
for column in df.columns:
    if (df[column].dtype == "int64" or df[column].dtype == "float64"):
        median = df[column].median()
        df[column] = df[column].fillna(median)
    else:
        df = df.dropna(subset=[column])

Здесь происходит отчищение, то есть если столбец с числовыми значениями, то мы заменяем пропуски на медиану всего столбца. Если качественный значения - удаляем.

In [26]:
def normalize_object_data(series):
    frequency_mapping = {
        "Male": 0,
        "Female": 1,
        "Poor": 0,
        "Fair": 1,
        "Good": 2,
        "Excellent": 3,
        "Occasionally": 0,
        "Regularly": 1,
        "Non-smoker": 0,
        "Light": 1,
        "Heavy": 2,
        "No": 0,
        "Yes": 1,
        "High School": 0,
        "Bachelor": 1,
        "Master": 2,
        "PhD": 3,
        "Tech": 0,
        "Office": 1,
        "Labor": 2,
        "Unemployed": 3,
        "Service": 4,
        "Healthcare": 5,
        "Vegan": 0,
        "Vegetarian": 1,
        "Omnivore": 2,
        "Keto": 3,
        "Strength": 0,
        "Cardio": 1,
        "Mixed": 2,
        "Low": 0,
        "Moderate": 1,
        "High": 2,
        "diseased": 0,
        "healthy": 1
    }
    series = series.map(frequency_mapping)
    return series

Это функция, которая позволяет нормализовать качественные значения, то есть Все значения в столбце меняются на 0, 1

In [27]:
#Normalization of Data Frame
array_normalize = []
for column in df.columns:
    if (df[column].dtype == "int64" or df[column].dtype == "float64"):
        array = np.array(df[column])
        normalize_arr = preprocessing.normalize([array])
        array_normalize.append(normalize_arr[0])
    else:
        normalize_column = normalize_object_data(df[column])
        normalize_arr = np.array(normalize_column)
        array_normalize.append(normalize_arr)


Здесь соответсвенно идет нормализация всех данных в Data Frame, числовые - с помощью preprocessing.normalize, а качественные через предыдущую функцию

In [28]:
#Delete dublicates
df_final = pd.DataFrame(array_normalize)
df_final = df_final.T
#insurance_data_unique = insurance_data.drop_duplicates()
df_final = df_final.drop_duplicates()
df_final.columns = df.columns


Поскольку мы сохранили нормализованные данные в двумерный массив, то теперь мы из него делаем обратно Data Frame, транспонируем таблицу(чтобы столбцы и строки были на своих местах а не наоборот), удаляем дубликаты. Также меняем названия столбцов на исходные.

In [29]:
#Diagram
# plt.figure(figsize=(10, 7))
# plt.scatter()